In [2]:
!pip install gdown

# Replace 'YOUR_FILE_ID' with the actual file ID from the shareable link
file_id = '183-vEE-6K08JVeFlHsOnP3kwIeDHGU4U'
file_url = f'https://drive.google.com/uc?id={file_id}'

# Download the file using gdown
!gdown "$file_url"

# Unzip the downloaded file



Downloading...
From: https://drive.google.com/uc?id=183-vEE-6K08JVeFlHsOnP3kwIeDHGU4U
To: /content/train.zip
100% 957M/957M [00:09<00:00, 102MB/s]


In [3]:
!unzip train.zip

Streaming output truncated to the last 5000 lines.
  inflating: train/images/6131.jpg   
  inflating: train/images/1516.jpg   
  inflating: train/images/4137.jpg   
  inflating: train/images/5054.jpg   
  inflating: train/images/2760.jpg   
  inflating: train/images/48.jpg     
  inflating: train/images/3671.jpg   
  inflating: train/images/2593.jpg   
  inflating: train/images/5915.jpg   
  inflating: train/images/4303.jpg   
  inflating: train/images/2573.jpg   
  inflating: train/images/6860.jpg   
  inflating: train/images/3534.jpg   
  inflating: train/images/6475.jpg   
  inflating: train/images/6165.jpg   
  inflating: train/images/7116.jpg   
  inflating: train/images/4127.jpg   
  inflating: train/images/2913.jpg   
  inflating: train/images/560.jpg    
  inflating: train/images/3501.jpg   
  inflating: train/images/3069.jpg   
  inflating: train/images/3103.jpg   
  inflating: train/images/2906.jpg   
  inflating: train/images/3666.jpg   
  inflating: train/images/2514.jpg   

In [4]:
!pip install efficientnet_pytorch

  Preparing metadata (setup.py) ... done
  Created wheel for efficientnet_pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16428 sha256=da0e21830a3d4c5b8fc15ca1e8a13709cd9f5bfd6335d9a469e2eca70e907420
  Stored in directory: /root/.cache/pip/wheels/03/3f/e9/911b1bc46869644912bda90a56bcf7b960f20b5187feea3baf
Successfully built efficientnet_pytorch


In [5]:
import torch
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from efficientnet_pytorch import EfficientNet
import os
import pandas as pd
from PIL import Image
from tqdm.auto import tqdm
from torch.optim.lr_scheduler import ExponentialLR
from torch.cuda.amp import autocast, GradScaler

# Parameters
IMG_SIZE = 244
PATH = 'model_checkpoint.pth'  # Update with your desired path
NUM_CLASSES = 1000
NUM_WORKERS = 4
BATCH_SIZE = 32
LEARNING_RATE = 0.001
MOMENTUM = 0.9
WEIGHT_DECAY = 1e-4
NUM_EPOCHS = 10

# Define the ImageDataset class
class ImageDataset(Dataset):
    def __init__(self, df, img_dir, transform=None):
        self.df = df
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def load_image(self, img_path):
        img = Image.open(img_path).convert('RGB')
        img = img.resize((IMG_SIZE, IMG_SIZE))
        return img

    def __getitem__(self, idx):
        fname = self.df.loc[idx, 'filename']
        label = self.df.loc[idx, 'label']
        img_path = os.path.join(self.img_dir, fname)
        img = self.load_image(img_path)
        if self.transform:
            img = self.transform(img)  # Convert PIL to Tensor
        return img, label

# Load Dataframes
train_df = pd.read_csv('/content/train/train.csv')

# Dataset & Dataloader
train_transform = transforms.Compose([
    transforms.ToTensor()])
train_ds = ImageDataset(train_df, '/content/train/images', transform=train_transform)
train_dl = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = EfficientNet.from_pretrained('efficientnet-b0', num_classes=NUM_CLASSES).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
scheduler = ExponentialLR(optimizer, gamma=0.1)

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b0-355c32eb.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b0-355c32eb.pth
100%|██████████| 20.4M/20.4M [00:00<00:00, 155MB/s]


Loaded pretrained weights for efficientnet-b0


In [7]:

# Gradient scaling
scaler = GradScaler()

# Training loop
pbar = tqdm(total=NUM_EPOCHS, desc="Training")
for epoch in range(NUM_EPOCHS):
    scheduler.step()
    epoch_loss = 0.0
    correct_predictions = 0
    total_samples = 0
    pbar_iterations = tqdm(total=len(train_dl), desc=f"Epoch {epoch + 1}/{NUM_EPOCHS}", leave=True)


    for X_batch, y_batch in train_dl:
        with autocast():

            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            y_pred = model(X_batch)


            loss = torch.nn.functional.cross_entropy(y_pred, y_batch)
            epoch_loss += loss.item()



        # Calculate accuracy
        _, predicted_labels = torch.max(y_pred, 1)
        correct_predictions += (predicted_labels == y_batch).sum().item()
        total_samples += y_batch.size(0)


        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        optimizer.zero_grad()
        pbar_iterations.update(1)

    pbar_iterations.close()
    optimizer.step()
    scheduler.step()

    avg_loss = epoch_loss / len(train_dl)
    accuracy = correct_predictions / total_samples
    print(f'Epoch [{epoch+1}/{NUM_EPOCHS}], Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}')
    pbar.set_postfix(train_acc=accuracy)
    torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': avg_loss,
            'accuracy': accuracy
     },PATH)

# Close the tqdm progress bar
pbar.close()

Training:   0%|          | 0/10 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


Epoch 1/10:   0%|          | 0/225 [00:00<?, ?it/s]

Epoch [1/10], Loss: 2.0180, Accuracy: 0.5739


Epoch 2/10:   0%|          | 0/225 [00:00<?, ?it/s]

...

Epoch [8/10], Loss: 0.5334, Accuracy: 0.8181


Epoch 9/10:   0%|          | 0/225 [00:00<?, ?it/s]

Epoch [9/10], Loss: 0.5395, Accuracy: 0.8178


Epoch 10/10:   0%|          | 0/225 [00:00<?, ?it/s]

Epoch [10/10], Loss: 0.5284, Accuracy: 0.8224
